In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

import warnings
import logging
logging.disable(logging.CRITICAL)
from tqdm.autonotebook import tqdm

import torch
from torch.nn import MSELoss, CrossEntropyLoss
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
import pytorch_lightning as pl

import optuna
from optuna.integration import PyTorchLightningPruningCallback

from darts import TimeSeries, concatenate
from darts.models import NHiTSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, rmse, mse

from sklearn.model_selection import train_test_split

In [ ]:
%run ../utils/preprocessing.ipynb
%run ../utils/losses.ipynb

In [ ]:
TICKERS = ['QCOM', 'NVDA', 'AMZN', 'MSFT', 'GOOG', 'TSLA', 'AMD', 'INTC', 'NFLX', 'BAC', 'WFC', 'GS', 'MA', 'SQ', 'PYPL']
frequency = 15

model_type = 'TFT'

results_df = pd.DataFrame(columns=['Model', 'Frequency', 'Ticker', 'RMSE', 'MSE', 'MAPE'])

In [ ]:
for ticker in TICKERS:
    DATA_FREQUENCY = minute_frequencies_conventions[frequency]

    MODEL_NAME = f'{ticker}_{frequency}_TFT'

    dfY = pd.read_csv(f'saved_models/{frequency}/{MODEL_NAME}/backtesting_dataftame.csv')
    
    rmse = rmse_loss(dfY['mid_close'], dfY['prediction'])
    mse = mse_loss(dfY['mid_close'], dfY['prediction'])
    mape = mape_loss(dfY['mid_close'], dfY['prediction'])

    results_df = pd.concat([results_df, pd.DataFrame([{'Model': model_type, 'Frequency': f'{frequency}M', 'Ticker': ticker, 'RMSE': rmse, 'MSE': mse, 'MAPE': mape}])], ignore_index=True)

In [ ]:
results_df.to_csv(f'./saved_models/{frequency}T_losses.csv')